In [1]:
import torch
import numpy as np
from stable_baselines3 import DQN

import os, shutil, sys
sys.path.append(os.path.abspath('./env'))
sys.path.append(os.path.abspath('./common'))


from env_find_path import FindPath

import common.mlflow_sb3_helper as mlf
import common.tensorboard_sb3_helper as tnb
USE_MLFLOW = False

from pytz import timezone
from datetime import datetime

TZ = timezone('Europe/Moscow')

activations = {'ReLU':torch.nn.ReLU, 'Tanh':torch.nn.Tanh}
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
log_helper = {}
log_path = {}

if USE_MLFLOW:
    log_helper = mlf.MLflowServerHelper("http://192.168.0.206:2670", False)
else:
    log_helper = tnb.TensorboardHelper("",False)

In [3]:
option = {'start_radius': 5}
env = FindPath(target_point_rand=False, observation_render=True, area_size=200, options=option)
env_render = FindPath(target_point_rand=False, observation_render=True, render_mode='rgb_array',  area_size=200, options=option)

In [3]:
exp_params = {
    'env_name': '',
    'algorithm_name': '',
    'exp_id': 0,
    'exp_name': f'exp_{datetime.now(TZ).strftime("%d%m_%H%M%S")}',
    'seed': 21,
    'net': {
        'activation': 'ReLU',
        'qf': [256, 64],
    },
    'training': {
        'iteration_count': 3,
        'episode_count': 199000,
        'policy': 'CnnPolicy',
        'learning_rate': 0.001,
        'buffer_size': 10000,
        'learning_starts': 100,
        'batch_size': 256,
        'tau': 0.005,
        'gamma': 0.95,
        'verbose': 0,
        'device': device,
        'DQN':{
            'fraction_eps' : 0.8,
            'start_eps' : 1.0,
            'final_eps' : 0.3,
        },
    },
    'validation': {
        'validate_agent_every_n_eps': 60000,
        'log_interval': 10
    },
}

policy_kwargs = dict(activation_fn = activations[exp_params['net']['activation']],
                     net_arch = exp_params['net']['qf']
)




In [4]:
eps_var ={        
#'DQN1':{'fraction_eps' : 0.8, 'start_eps' : 1.0,'final_eps' : 0.3,},
'DQN2':{'fraction_eps' : 0.4, 'start_eps' : 1.0,'final_eps' : 0.3,},
#'DQN3':{'fraction_eps' : 0.1, 'start_eps' : 1.0,'final_eps' : 0.3,},
#'DQN4':{'fraction_eps' : 0.1, 'start_eps' : 1.0,'final_eps' : 0.1,},
'DQN5':{'fraction_eps' : 0.1, 'start_eps' : 0.3,'final_eps' : 0.05,},
'DQN6':{'fraction_eps' : 0., 'start_eps' : 0.05,'final_eps' : 0.05,},
}

In [ ]:
#!!!!! Новый эксперимент
exp_params['env_name'] = env.name()
exp_name = 'env_' + exp_params['env_name'] + '_' + exp_params['exp_name']
experiment_id = log_helper.new_experiment(exp_name)

In [ ]:
#!!!!! Дорасчет старого эксперимента
exp_params['env_name'] = env.name()
experiment_id = 161
exp_name = log_helper.get_experiment(experiment_id)
print(exp_name)

In [ ]:
exp_params['exp_id'] = experiment_id

b_first = True
name = "DQN"

for name, eps in eps_var.items():

    exp_params['algorithm_name'] = name
    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))
    exp_params['training']['DQN'] = eps
    
    model = None
    if b_first == True:
        b_first = False
        model = DQN(  exp_params['training']['policy'], 
                    env,
                    policy_kwargs=policy_kwargs,
                    learning_rate=exp_params['training']['learning_rate'],
                    buffer_size=exp_params['training']['buffer_size'],
                    learning_starts=exp_params['training']['learning_starts'],
                    batch_size=exp_params['training']['batch_size'],
                    tau=exp_params['training']['tau'],
                    gamma=exp_params['training']['gamma'],
                    verbose=exp_params['training']['verbose'],
                    exploration_fraction=exp_params['training']['DQN']['fraction_eps'],
                    exploration_initial_eps= exp_params['training']['DQN']['start_eps'],
                    exploration_final_eps= exp_params['training']['DQN']['final_eps'],
                    device=device,
                    )
    else:
        model = DQN.load(f'{exp_name}/model.zip', env=env, device=device)
        model.exploration_fraction=exp_params['training']['DQN']['fraction_eps']
        model.exploration_initial_eps= exp_params['training']['DQN']['start_eps']
        model.exploration_final_eps= exp_params['training']['DQN']['final_eps']
        model._setup_model()

    art_loc, exp_name, run_id = log_helper.learn_and_fix(
                                model=model, 
                                env= env_render,
                                run_name=name,
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'],
                                video_episods=1,
                                video_fps=10
                                )
    log_path[name] = f'{art_loc}/{run_id}/artifacts/{exp_name}/sb3/model.zip'


In [ ]:
log_path

In [5]:
experiment_id = 511
log_path = { 
'DQN': f'mlflow-artifacts:/{experiment_id}/f71b693d127d4ce28fca90a40231bd69/artifacts/env_FindPathCNN_exp_2610_160813/sb3/model.zip'
}

In [ ]:
load_path = os.path.join(str(experiment_id))
if  os.path.isdir(load_path) == False:
    os.mkdir(load_path)
load_path

In [ ]:

loadel_models = {}

for model_name, path in log_path.items():
    loadel_models[model_name] =  f'{load_path}/{model_name}/'
    log_helper.load_artifact(path,loadel_models[model_name])

In [8]:
option = {'start_radius': 2}
env = FindPath(target_point_rand=True, observation_render=True, area_size=200, options=option)
env_render = FindPath(target_point_rand=True, observation_render=True, render_mode='rgb_array',  area_size=200, options=option)

In [ ]:
#!!! Дорасчет в НОВОМ эксперименте
exp_params['env_name'] = env.name()
exp_params['exp_name'] = f'exp_{datetime.now(TZ).strftime("%d%m_%H%M%S")}'
exp_name = 'env_' + exp_params['env_name'] + '_' + exp_params['exp_name']
experiment_id = log_helper.new_experiment(exp_name)
exp_params['exp_id'] = experiment_id
print(exp_name)


In [ ]:
#!!! Дорасчет в СТАРОМ эксперименте

exp_name = log_helper.get_experiment(experiment_id)
print(exp_name)

In [ ]:
name = 'DQN'
path = loadel_models[name]
exp_params['algorithm_name'] = name
exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))
exp_params['training']['DQN'] = eps_var['DQN2']

b_first = True
model = None

for i in range(exp_params['training']['iteration_count']):

    if b_first == True:
        b_first = False
        model = DQN.load(f'{path}/model.zip', env=env, device=device)
        model.exploration_fraction=exp_params['training']['DQN']['fraction_eps']
        model.exploration_initial_eps= exp_params['training']['DQN']['start_eps']
        model.exploration_final_eps= exp_params['training']['DQN']['final_eps']
        model._setup_model()
    else:
        print(exp_name)
        model = DQN.load(f'{exp_name}/model.zip', env=env, device=device)


    art_loc, exp_name, run_id = log_helper.learn_and_fix(
                                model=model, 
                                env= env_render,
                                run_name=f'{name}_{i+4}',
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'],
                                video_episods=3,
                                video_fps=10
                                )
    


In [ ]:
name = "DQN"
path = loadel_models[name]
exp_params['algorithm_name'] = name
exp_params['exp_id'] = experiment_id

b_first = True

for name, eps in eps_var.items():

    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))
    exp_params['training']['DQN'] = eps
    
    model = None
    if b_first == True:
        b_first = False
        model = DQN.load(f'{path}/model.zip', env=env, device=device)
    else:
        model = DQN.load(f'{exp_name}/model.zip', env=env, device=device)

    model.exploration_fraction=exp_params['training']['DQN']['fraction_eps']
    model.exploration_initial_eps= exp_params['training']['DQN']['start_eps']
    model.exploration_final_eps= exp_params['training']['DQN']['final_eps']
    model._setup_model()

    art_loc, exp_name, run_id = log_helper.learn_and_fix(
                                model=model, 
                                env= env_render,
                                run_name=name,
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'],
                                video_episods=3,
                                video_fps=10
                                )
    log_path[name] = f'{art_loc}/{run_id}/artifacts/{exp_name}/sb3/model.zip'

In [ ]:
experiment_id

In [11]:
shutil.rmtree(os.path.join(load_path))